# Sistema de Recomendación

In [39]:
import pandas as pd
import numpy as np
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import nltk as nlt
from fastapi import HTTPException

In [2]:
df = pd.read_parquet(r'C:\Users\GASTON\Desktop\PI1\DATA\df_mlops.parquet')
#Creamos un DataFrame a partir del dataset previamente depurado

In [3]:
df.head(3)

,id,spoken_languagesname,genresname,title,overview,vote_average,status,production_countriesname,release_year,director,actor
2,10001,English,"Science Fiction, Comedy",Young Einstein,Albert Einstein is the son of a Tasmanian appl...,4.5,Released,Australia,1988,Yahoo Serious,"Yahoo Serious, Odile Le Clezio, Peewee Wilson,..."
3,100010,English,"Drama, War",Flight Command,"A rookie flyer, Ens. Alan Drake, joins the fam...",6.0,Released,United States of America,1940,Frank Borzage,"Robert Taylor, Ruth Hussey, Walter Pidgeon, Pa..."
4,100017,Deutsch,Drama,Hounded,Deals with the obsessive relationship between ...,4.8,Released,Germany,2006,Angelina Maccarone,"Kostja Ullmann, Maren Kroymann, Moritz Grove, ..."


In [4]:
#Como primer paso, debemos limpiar y tokenizar el texto.
#El primer elemento a utilizar para obtener una cadena de palabras clave, serán las stopwords (identificando las palabras adicionales).
# Se definirán las stopwords para los idiomas de inglés y español, convirtiendolas en sets

nlt.download('stopwords')

en_stopwords = set(stopwords.words('english'))
esp_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GASTON/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [14]:
# Función para limpiar y tokenizar nuestras columnas
def limpiarTexto(cadena):
    '''
    Esta función tokeniza el texto, elimina signos de puntuación y convierte los caracteres en minúsculas.
    Luego, filtra las stopwords.
    Espera como parámetro una cadena de texto.
    '''
    # Tokenización
    etiquetas = word_tokenize(cadena) #se convierte la cadena en una lista de palabras
    etiquetas = [word.lower() for word in etiquetas if word.isalpha()] # Se eliminan  signos de puntuación y convierte todo a minúsculas con lower()
    etiquetas = [word for word in etiquetas if word not in en_stopwords and word not in esp_stopwords] # Se quitan stopwords en inglés y español descargadas en el paso anterior
    return etiquetas

In [10]:
#import nltk
#nltk.data.path.append('C:/Users/GASTON/AppData/Roaming/nltk_data')

In [11]:
#import nltk
#nltk.data.path = ['C:/Users/GASTON/AppData/Roaming/nltk_data']
#nltk.download('punkt', download_dir='C:/Users/GASTON/AppData/Roaming/nltk_data')
#Fue necesario hacer estas modificaciones para que funcione la galería nltk


[nltk_data] Downloading package punkt to
[nltk_data]     C:/Users/GASTON/AppData/Roaming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
df['title_depurado'] = df['title'].apply(lambda x: limpiarTexto(x))
df['overview_depurado'] = df['overview'].dropna().apply(lambda x: limpiarTexto(x))
df.head(3)

,id,spoken_languagesname,genresname,title,overview,vote_average,status,production_countriesname,release_year,director,actor,title_depurado,overview_depurado
2,10001,English,"Science Fiction, Comedy",Young Einstein,Albert Einstein is the son of a Tasmanian appl...,4.5,Released,Australia,1988,Yahoo Serious,"Yahoo Serious, Odile Le Clezio, Peewee Wilson,...","[young, einstein]","[albert, einstein, tasmanian, apple, farmer, d..."
3,100010,English,"Drama, War",Flight Command,"A rookie flyer, Ens. Alan Drake, joins the fam...",6.0,Released,United States of America,1940,Frank Borzage,"Robert Taylor, Ruth Hussey, Walter Pidgeon, Pa...","[flight, command]","[rookie, flyer, ens, alan, drake, joins, famou..."
4,100017,Deutsch,Drama,Hounded,Deals with the obsessive relationship between ...,4.8,Released,Germany,2006,Angelina Maccarone,"Kostja Ullmann, Maren Kroymann, Moritz Grove, ...",[hounded],"[deals, obsessive, relationship, confused, tee..."


In [16]:
df = df.drop(['overview'],axis=1) #La columna overview (Pequeño resumen de la película) ha sido depurada y agregadas
#sus palabras clave a overview_depurado, por lo que se puede eliminar. No así el título, que será el retorno del 
#sistema de recomendación.

In [19]:
# La columna overview_depurado presenta algunos campos que no son strings. 
# Por eso, convertimos esa columna, la de título, título depurado y la de género en texto.
df['overview_depurado'] = df['overview_depurado'].astype(str)
df['title'] = df['title'].astype(str)
df['title_depurado'] = df['title_depurado'].astype(str)
df['genresname'] = df['genresname'].astype(str)

Ahora que se cuenta con los campos específicos, es preciso generar una columna que contenga la información de todos ellos. Esto simplificará la vectorización y análisis (en comparación a trabajar con varias columnas).
Se busca lograr una representación completa de la película, agregando las palabras clave del título, género y descripción. Además, se ponderará o dará mayor valor al título y al género, para que tengan más importancia, destacándose en la matriz TF-IDF para mejorar la precisión de las recomendaciones.

In [20]:
df['descripcion_combinada'] = df['title_depurado']+ ' '+ df['title_depurado'] + ' ' + df['overview_depurado'] + ' ' + df['genresname']+ ' ' + df['genresname']+ ' ' + df['genresname']
#Se crea la columna "descripción combinada", que contiene los valores del título (depurado), el resumen (depurado) y el género.
# Como se dijo, se repiten los valores del título y género para que tengan mayor fuerza.

In [21]:
# Para realizar la vectorización, creamos una lista de stopwords de los dos idiomas
stopwords_todas = list(set(en_stopwords).union(set(esp_stopwords)))

Ahora que tenemos todos los valores, debemos vectorizar y calcular la similitud del coseno para definir nuestra función de recomendación.
La vectorización se realiza con TFIDFVectorizer, con todas las stopwords guardadas.

In [25]:
# Vectorización utilizando TF-IDF con stop words en inglés y español
vectorizacion = TfidfVectorizer(stop_words=stopwords_todas, max_features=5000) #Se definen los paráemetros para convertir el texto en una matriz.
matrizTFIDF = vectorizacion.fit_transform(df['descripcion_combinada']) #Se crea una matriz TF-IDF sobre la columna "descripción combinada"
print(matrizTFIDF.shape) #dimensiones de la matriz

(34318, 5000)


In [26]:
# Realizamos la reducción de dimensionalidad
val_singulares = TruncatedSVD(n_components=100) #Se reduce la dimensionalidad de la matriz a 100 dimensiones, basándose en los valores singulares más altos.
matriz_sing = val_singulares.fit_transform(matrizTFIDF)#Aplica el ajuste y la transformación a la matriz TF-IDF
print(matriz_sing.shape) #Nuevas dimensiones sobre los valores singulares más importantes

(34318, 100)


La reducción de la dimensionalidad genera una notable mejora de eficacia. Al eliminarse los componentes principales, se eliminan los menos significativos, que podrían introducir ruido al análisis.
Además, la menor dimensionalidad contribuye con la mejora del rendimiento y la capacidad de generalización del modelo.

In [27]:
# Ahora debemos calcular la similitud del coseno entre las filas de la matriz reducida (esto permitirá encontrar
#  películas similares en función de sus características combinadas: título, descripción y género). 
# Además, permite analizar relaciones entre películas, lo que ayuda a mejorar la precisión del sistema de recomendación.

similitud_coseno = cosine_similarity(matriz_sing, matriz_sing) #Para buscar la similitud, cruzamos dos veces la matriz.

Ahora estamos en condiciones de crear la función de recomendación.

In [48]:
def recomendacion(title, df=df, similitud_coseno=similitud_coseno):
    try:
        indice = df[df['title'].str.lower() == title.lower()].index[0] # Se define el índice de película similar
        similitud = list(enumerate(similitud_coseno[indice])) 
        similitud = sorted(similitud, key=lambda x: x[1], reverse=True) #Se obtiene el score de similitud y se ordenan.
        # Se obtienen las películas similares, dejando afuera la primera(que es el mismo título):
        similitud = similitud[1:6]
        indices_similares = [i[0] for i in similitud]
        peliculas_similares = df['title'].iloc[indices_similares].tolist() #Se crea una lista con las películas
        mensaje = f"Se encontraron los siguientes títulos similares a '{title}':"
        print(mensaje)
        # Retornar la lista de títulos similares
        return peliculas_similares
    except IndexError:
        print(f"No se encontró la película '{title}'. Por favor, intente con otro título.")

In [49]:
test = recomendacion('toy story')
test

Se encontraron los siguientes títulos similares a 'toy story':


['Toy Story 2', 'Botsman i Popugay', 'Banana', 'Feed the Kitty', 'Hop']

In [50]:
test2 = recomendacion('Young Einstein')
test2

Se encontraron los siguientes títulos similares a 'Young Einstein':


['Easy on the Eyes',
 'Hibernatus',
 'Young Frankenstein',
 'Simon',
 'Cabbage Soup']

In [51]:
test3= recomendacion('Star Wars')
test3

Se encontraron los siguientes títulos similares a 'Star Wars':


['Star Trek Beyond',
 'Battle of the Damned',
 'Power Rangers',
 'Wheels of Fire',
 'Robot Overlords']

In [52]:
test_error = recomendacion('askedddego')
test_error

No se encontró la película 'askedddego'. Por favor, intente con otro título.


Vamos a reducir el dataset sólo a los elementos necesarios para el sistema de recomendación.

In [53]:
df.columns

Index(['id', 'spoken_languagesname', 'genresname', 'title', 'vote_average',
       'status', 'production_countriesname', 'release_year', 'director',
       'actor', 'title_depurado', 'overview_depurado',
       'descripcion_combinada'],
      dtype='object')

In [54]:
columnas_out = ['spoken_languagesname', 'vote_average', 'status', 'production_countriesname', 'release_year', 'director', 'actor']

In [58]:
df

,id,genresname,title,title_depurado,overview_depurado,descripcion_combinada
0,10001,"Science Fiction, Comedy",Young Einstein,"['young', 'einstein']","['albert', 'einstein', 'tasmanian', 'apple', '...","['young', 'einstein'] ['young', 'einstein'] ['..."
1,100010,"Drama, War",Flight Command,"['flight', 'command']","['rookie', 'flyer', 'ens', 'alan', 'drake', 'j...","['flight', 'command'] ['flight', 'command'] ['..."
2,100017,Drama,Hounded,['hounded'],"['deals', 'obsessive', 'relationship', 'confus...","['hounded'] ['hounded'] ['deals', 'obsessive',..."
3,10002,"Drama, Crime, Romance",Mona Lisa,"['mona', 'lisa']","['george', 'released', 'prison', 'manages', 'g...","['mona', 'lisa'] ['mona', 'lisa'] ['george', '..."
4,100024,"Thriller, Horror",Bloodwork,['bloodwork'],"['couple', 'college', 'students', 'decide', 's...","['bloodwork'] ['bloodwork'] ['couple', 'colleg..."
...,...,...,...,...,...,...
34313,99946,Comedy,Exit Smiling,"['exit', 'smiling']","['beatrice', 'lillie', 'jack', 'pickford', 'st...","['exit', 'smiling'] ['exit', 'smiling'] ['beat..."
34314,9995,"Drama, Action, Crime",Turn It Up,['turn'],"['trying', 'bootstrap', 'way', 'brooklyn', 'me...","['turn'] ['turn'] ['trying', 'bootstrap', 'way..."
34315,9997,"Fantasy, Action, Science Fiction, Horror",Gabriel,['gabriel'],"['gabriel', 'tells', 'story', 'archangel', 'fi...","['gabriel'] ['gabriel'] ['gabriel', 'tells', '..."
34316,99977,Comedy,Hot Stuff,"['hot', 'stuff']","['police', 'department', 'burglary', 'task', '...","['hot', 'stuff'] ['hot', 'stuff'] ['police', '..."


In [59]:
def recomendacion(title, df=df, similitud_coseno=similitud_coseno):
    try:
        indice = df[df['title'].str.lower() == title.lower()].index[0] # Se define el índice de película similar
        similitud = list(enumerate(similitud_coseno[indice])) 
        similitud = sorted(similitud, key=lambda x: x[1], reverse=True) #Se obtiene el score de similitud y se ordenan.
        # Se obtienen las películas similares, dejando afuera la primera(que es el mismo título):
        similitud = similitud[1:6]
        indices_similares = [i[0] for i in similitud]
        peliculas_similares = df['title'].iloc[indices_similares].tolist() #Se crea una lista con las películas
        mensaje = f"Se encontraron los siguientes títulos similares a '{title}':"
        print(mensaje)
        # Retornar la lista de títulos similares
        return peliculas_similares
    except IndexError:
        print(f"No se encontró la película '{title}'. Por favor, intente con otro título.")

In [60]:
test4= recomendacion('Star Wars')
test4

Se encontraron los siguientes títulos similares a 'Star Wars':


['Star Trek Beyond',
 'Battle of the Damned',
 'Power Rangers',
 'Wheels of Fire',
 'Robot Overlords']

In [61]:
test_error2=recomendacion('asdasd')
test_error2

No se encontró la película 'asdasd'. Por favor, intente con otro título.


In [63]:
df.to_parquet(r'C:\Users\GASTON\Desktop\PI1\DATArecomendacion.parquet',index=True)

In [64]:
prueba = pd.read_parquet(r'C:\Users\GASTON\Desktop\PI1\DATArecomendacion.parquet')
prueba.head()

,id,genresname,title,title_depurado,overview_depurado,descripcion_combinada
0,10001,"Science Fiction, Comedy",Young Einstein,"['young', 'einstein']","['albert', 'einstein', 'tasmanian', 'apple', '...","['young', 'einstein'] ['young', 'einstein'] ['..."
1,100010,"Drama, War",Flight Command,"['flight', 'command']","['rookie', 'flyer', 'ens', 'alan', 'drake', 'j...","['flight', 'command'] ['flight', 'command'] ['..."
2,100017,Drama,Hounded,['hounded'],"['deals', 'obsessive', 'relationship', 'confus...","['hounded'] ['hounded'] ['deals', 'obsessive',..."
3,10002,"Drama, Crime, Romance",Mona Lisa,"['mona', 'lisa']","['george', 'released', 'prison', 'manages', 'g...","['mona', 'lisa'] ['mona', 'lisa'] ['george', '..."
4,100024,"Thriller, Horror",Bloodwork,['bloodwork'],"['couple', 'college', 'students', 'decide', 's...","['bloodwork'] ['bloodwork'] ['couple', 'colleg..."
